In [1]:
%pip install ultralytics opencv-python face-recognition

Note: you may need to restart the kernel to use updated packages.Collecting ultralytics

  Using cached ultralytics-8.2.90-py3-none-any.whl.metadata (41 kB)
  Using cached face_recognition-1.3.0-py2.py3-none-any.whl.metadata (21 kB)
  Using cached matplotlib-3.7.5-cp38-cp38-win_amd64.whl.metadata (5.8 kB)
  Using cached torch-2.4.1-cp38-cp38-win_amd64.whl.metadata (27 kB)
  Using cached torchvision-0.19.1-cp38-cp38-win_amd64.whl.metadata (6.1 kB)
  Using cached py_cpuinfo-9.0.0-py3-none-any.whl.metadata (794 bytes)
  Using cached pandas-2.0.3-cp38-cp38-win_amd64.whl.metadata (18 kB)
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached ultralytics_thop-2.0.6-py3-none-any.whl.metadata (9.1 kB)
  Using cached face_recognition_models-0.3.0.tar.gz (100.1 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached dlib-19.24.6.tar.gz (3.4 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [41 lines of output]
      running bdist_wheel
      running build
      running build_ext
      
      
                         CMake is not installed on your system!
      
          Or it is possible some broken copy of cmake is installed on your system.
          It is unfortunately very common for python package managers to include
          broken copies of cmake.  So if the error above this refers to some file
          path to a cmake file inside a python or anaconda or miniconda path then you
          should delete that broken copy of cmake from your computer.
      
          Instead, please get an official copy of cmake from one of these known good
          sources of an official cmake:
              - cmake.org (this is how windows users should get cmake)
              - apt install cmake (for Ubuntu or Debian based systems)
              - yum instal

In [2]:
from ultralytics import YOLO

# Tải mô hình YOLOv8
model = YOLO("yolov8n.pt")

ModuleNotFoundError: No module named 'ultralytics'

In [ ]:
import cv2
import face_recognition

known_encodings = []
known_names = []


# Hàm để phát hiện khuôn mặt sử dụng YOLOv8
def detect_faces(image):
    results = model(image)
    boxes = results[0].boxes.xyxy  # Lấy toạ độ khung hình các khuôn mặt
    return boxes

# Hàm nhận diện khuôn mặt
def recognize_faces(known_encodings, frame):
    face_locations = detect_faces(frame)
    face_encodings = face_recognition.face_encodings(frame, face_locations)
    
    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        matches = face_recognition.compare_faces(known_encodings, face_encoding)
        name = "Unknown"

        if True in matches:
            first_match_index = matches.index(True)
            name = known_names[first_match_index]

        # Vẽ khung hình xung quanh khuôn mặt và hiển thị tên
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.putText(frame, name, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 0, 0), 2)
    
    return frame

In [ ]:
# Giả sử bạn có hình ảnh khuôn mặt nhân viên trong một thư mục
import os

for file in os.listdir("employees_photos"):
    image = face_recognition.load_image_file(f"employees_photos/{file}")
    encoding = face_recognition.face_encodings(image)[0]
    known_encodings.append(encoding)
    known_names.append(os.path.splitext(file)[0])  # Tên người được nhận diện

In [ ]:
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = recognize_faces(known_encodings, frame)
    
    cv2.imshow("Face Recognition Attendance", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
import csv
from datetime import datetime

# Hàm ghi kết quả điểm danh
def save_attendance(name):
    with open('attendance.csv', mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([name, datetime.now().strftime("%Y-%m-%d %H:%M:%S")])
